# Team members:
- Nguyễn Duy Minh (myself)
- Vũ Trường Giang
- Vũ Hồng Phúc

In [171]:
import pandas as pd
import statsmodels.api as sm
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [172]:
df = pd.read_csv(r'./owid-2022-clean.csv')

In [173]:
# Drop irrelevant fields
print(df.columns.shape)
df.drop(["location", "date", "total_deaths_per_million", "total_cases_per_million", "population", "tests_units"], axis=1, inplace=True)

print(df.columns.shape)

(31,)
(25,)


 # 1. Data preparations
 - NaN/missing values are replaced with the median value of the country's other records.
 - If all values are NaN/missing/zeros, replace them with the median value of the records from all countries within the same continent
 - Only replace zero values when the entire column of the country is 0. This is because countries like Vietnam have days where no deaths are recorded, but other days still record a positive number. This is different to countries where no death numbers are recorded. There are cases where zero values may slightly raise some eyebrows, for example Albania recording no new cases in multiple days. However, they might as well be zero because "new_cases_per_million" mean some thing like "new_cases_reported_per_million" for example. Since our target variables are also based on reports, it may be best to simply also respect these reported zero values in other columns.
 - A much better way to deal with these values (in my opinion) is to group countries by factors like GDP and poverty rate. However, it is more difficult and time consuming to do this since multiple factors need to be considered and the values are numeric instead of categorical. Since we have 6 deadlines this weekend, our team unfortunately had to abandon this approach for lack of time.


In [174]:
applied_cols = list(df.columns)
df_group_iso = df.groupby(["iso_code"])
df_group_cont = df.groupby(["continent"])

for col_name, col_data in df.iteritems():
    if col_name == "iso_code" or col_name=="continent":
        continue
    # replace nan with country median
    df[col_name] = df_group_iso[col_name].apply(lambda x: x.fillna(x.median()))
    # replace nan with continent median
    df[col_name] = df_group_cont[col_name].apply(lambda x: x.fillna(x.median()))

# Replace zeroes
df_group_cont_med = df_group_cont.median()
for col_name, col_data in df_group_iso:    
    for col in col_data:
        if (col_data[col] == 0).all():
            target = df.loc[df["iso_code"] == col_name]
            df.loc[df["iso_code"] == col_name, col] = target[col].replace(0, df_group_cont_med.at[target["continent"].iloc[0], col])



In [175]:
indicator_cols = pd.get_dummies(df[ "continent"])
df = pd.concat([df, indicator_cols], axis=1)
df = df.groupby("iso_code").agg("mean")
print(df.columns)

          new_cases_per_million  new_deaths_per_million  \
iso_code                                                  
AFG                   14.337571                0.100429   
AGO                    1.233429                0.004143   
ALB                  314.859857                2.038857   
AND                 1569.777571                3.693714   
ARE                  204.511000                0.314286   
...                         ...                     ...   
WSM                    0.713714                0.000000   
YEM                    1.157286                0.075143   
ZAF                   47.509571                2.208143   
ZMB                   17.139429                0.121000   
ZWE                    6.966714                0.227143   

          new_tests_smoothed_per_thousand  positive_rate  tests_per_case  \
iso_code                                                                   
AFG                              0.033000       0.452071        2.200000   
AGO 

In [176]:
df.to_csv("./data.csv")

In [177]:
def linear_regression(X: pd.DataFrame, y: pd.DataFrame, ft_in_use, log = False):
    X = X[ft_in_use]

    model = sm.OLS(y, sm.add_constant(X))
    results = model.fit()

    if (log):
        print(results.summary())

    y_pred = results.predict(sm.add_constant(X))
    print(f'MSE: {sum((y - y_pred) ** 2) / len(y)}')
    return results

In [178]:
df = pd.read_csv('data.csv')

ft_predict = ['new_deaths_per_million', 'new_cases_per_million']
ft_ignore = ['iso_code', 'location', 'date', 'total_deaths', 'total_cases_per_million', 'total_deaths_per_million', 'population', 'tests_units'] # add ignore column names here

features = list(set(df.columns).difference(ft_predict + ft_ignore))

y = df[ft_predict]
X = df[features]

In [179]:
def new_deaths_per_million(X: pd.DataFrame, y: pd.DataFrame, ft_in_use):
    return linear_regression(X, y['new_deaths_per_million'], ft_in_use, log = True)

In [180]:
def new_cases_per_million(X: pd.DataFrame, y: pd.DataFrame, ft_in_use):
    return linear_regression(X, y['new_cases_per_million'], ft_in_use, log = True)

In [181]:
# ordinary linear regression

dth = new_deaths_per_million(X, y, features)
cas = new_cases_per_million(X, y, features)

                              OLS Regression Results                              
Dep. Variable:     new_deaths_per_million   R-squared:                       0.554
Model:                                OLS   Adj. R-squared:                  0.482
Method:                     Least Squares   F-statistic:                     7.739
Date:                    Sat, 04 Jun 2022   Prob (F-statistic):           1.56e-17
Time:                            23:22:39   Log-Likelihood:                -403.13
No. Observations:                     189   AIC:                             860.3
Df Residuals:                         162   BIC:                             947.8
Df Model:                              26                                         
Covariance Type:                nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [182]:
# reorder columns according to p-values
# save to `features` variable

deaths_ft = sorted(features, key=lambda col: dth.pvalues[col])
cases_ft = sorted(features, key=lambda col: cas.pvalues[col])

alpha = 0.05

In [183]:
# stepwise forward regression

def forward_reg(title, features, func):
    if (title is not None):
        print(title)
    
    ft_in_use = []
    for col in features:
        ft_in_use.append(col)

        res = func(X, y, ft_in_use)

        print(f'Added ft:  {col}')
        print(f'Ft in use: {ft_in_use}')

        if (max(res.pvalues) > alpha):
            print(f'Break due to stopping rule: max pvalue = {max(res.pvalues)} > {alpha}')
            break

        print('*' * 20)

In [184]:
# stepwise forward regression
forward_reg('New deaths per million model:', deaths_ft, new_deaths_per_million)

New deaths per million model:
                              OLS Regression Results                              
Dep. Variable:     new_deaths_per_million   R-squared:                       0.335
Model:                                OLS   Adj. R-squared:                  0.331
Method:                     Least Squares   F-statistic:                     94.06
Date:                    Sat, 04 Jun 2022   Prob (F-statistic):           2.84e-18
Time:                            23:22:40   Log-Likelihood:                -440.92
No. Observations:                     189   AIC:                             885.8
Df Residuals:                         187   BIC:                             892.3
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

In [185]:
forward_reg('New cases per million model:', cases_ft, new_cases_per_million)

New cases per million model:
                              OLS Regression Results                             
Dep. Variable:     new_cases_per_million   R-squared:                       0.192
Model:                               OLS   Adj. R-squared:                  0.188
Method:                    Least Squares   F-statistic:                     44.57
Date:                   Sat, 04 Jun 2022   Prob (F-statistic):           2.71e-10
Time:                           23:22:40   Log-Likelihood:                -1621.1
No. Observations:                    189   AIC:                             3246.
Df Residuals:                        187   BIC:                             3253.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

In [186]:
# stepwise backward regression

def backward_reg(title, features, func):
    if (title is not None):
        print(title)
    
    features.reverse()
    ft_in_use = features.copy()
    for col in features:
        ft_in_use.remove(col)

        res = func(X, y, ft_in_use)

        print(f'Removed ft:  {col}')
        print(f'Ft in use: {ft_in_use}')

        if (max(res.pvalues) < alpha):
            print(f'Break due to stopping rule: max pvalue = {max(res.pvalues)} < {alpha}')
            break

        print('*' * 20)

In [187]:
# stepwise backward regression
backward_reg('New deaths per million model:', deaths_ft, new_deaths_per_million)

New deaths per million model:
                              OLS Regression Results                              
Dep. Variable:     new_deaths_per_million   R-squared:                       0.554
Model:                                OLS   Adj. R-squared:                  0.486
Method:                     Least Squares   F-statistic:                     8.097
Date:                    Sat, 04 Jun 2022   Prob (F-statistic):           5.41e-18
Time:                            23:22:40   Log-Likelihood:                -403.13
No. Observations:                     189   AIC:                             858.3
Df Residuals:                         163   BIC:                             942.5
Df Model:                              25                                         
Covariance Type:                nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [188]:
backward_reg('New cases per million model:', cases_ft, new_cases_per_million)

New cases per million model:
                              OLS Regression Results                             
Dep. Variable:     new_cases_per_million   R-squared:                       0.580
Model:                               OLS   Adj. R-squared:                  0.516
Method:                    Least Squares   F-statistic:                     9.019
Date:                   Sat, 04 Jun 2022   Prob (F-statistic):           6.25e-20
Time:                           23:22:41   Log-Likelihood:                -1559.2
No. Observations:                    189   AIC:                             3170.
Df Residuals:                        163   BIC:                             3255.
Df Model:                             25                                         
Covariance Type:               nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------